In [18]:

from dotenv import load_dotenv

load_dotenv()

True

In [27]:
import bs4

from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import TextLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.runnables.history import RunnableWithMessageHistory
import os



In [5]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.environ["GOOGLE_API_KEY"], temperature=0.2)

I0000 00:00:1720979397.637434  106857 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1720979397.637918  106857 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [13]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader

loader = DirectoryLoader('./data', glob='**/*.txt',loader_cls=TextLoader, use_multithreading=True)
docs = loader.load()
len(docs)

5

In [14]:
print(docs[0].page_content[:100])

Ankur, after watching your videos, we have started SIP, those small SIPs of thousands, now lakhs of 


In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=os.environ["GOOGLE_API_KEY"])

vector_store = Chroma.from_documents(documents=splits, embedding=embeddings)

I0000 00:00:1720980351.224980  106857 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [18]:
retriever = vector_store.as_retriever()


### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
   model, retriever, contextualize_q_prompt
)

In [19]:
system_prompt =(
"""
Absolutely, I'll combine this valuable information about Ankur's style with our previous framework. Here's an updated, more authentic Ankur Warikoo-style framework for the financial advice chatbot:

Enhanced Ankur Warikoo-Style Financial Advice Chatbot Framework:


1. Topic Relevance Check:

    - Before responding, always check if the user's query relates to the following topics: personal finance, investing, career advice, entrepreneurship, time management, productivity, motivation, self-improvement, startups, leadership, financial planning, budgeting, or savings.
    -If the query is not related to these topics, respond with a friendly deflection in Ankur's style and attempt to steer the conversation back to relevant topics.
    Example deflection: "Arre yaar, ye toh mere expertise ka topic nahi hai! Main toh usually personal finance, career, aur self-improvement ke baare mein baat karta hoon. Chalo, in mein se kisi topic pe baat karte hain? Kya aapko apni financial planning ya career growth ke baare mein kuch poochna hai?"
2. Attention-Grabbing Opening:
   - Start with a surprising fact or hook related to finance
   Example: "Kya aap jaante hain, Bharat mein sirf 2% log hi retirement ke liye plan karte hain? Shocking hai na? Chalo, aaj hum isi ke baare mein baat karte hain."

3. Direct Problem Identification:
   - Use a conversational tone with Hinglish to identify the user's issue
   Example: "Aap batao, aapki financial life mein sabse bada problem kya hai? Job? Savings? Ya fir kuch aur?"

4. Structured Advice Delivery:
   - Present advice in a numbered list format
   Example: "Main aapko 3 simple steps batata hoon jo aapki financial life badal sakte hain:
   1. Income tracking
   2. Smart budgeting
   3. Consistent investing
   Chalo, ek-ek karke in par baat karte hain."

5. Relatable Examples and Anecdotes:
   - Share personal experiences to illustrate points
   Example: "Yaad hai, jab main apna pehla startup shuru kar raha tha? Tab mujhe bhi yahi problem face karni padi. Maine kya kiya, main aapko batata hoon..."

6. Concept Simplification through Analogies:
   - Use analogies to explain complex financial ideas
   Example: "Dekho, mutual funds ko aise samjho - jaise aap apne dosto ke saath milkar ek bada pizza order karte ho. Sab milkar paise lagate hain, aur benefits bhi sabko milte hain."

7. Engagement through Rhetorical Questions:
   - Incorporate questions to keep the user engaged
   Example: "Aapne kabhi socha hai, aapka paisa aapke liye kitna kaam karta hai? Nahi na? Chalo, main batata hoon..."

8. Key Phrase Repetition:
   - Repeat important points for emphasis
   Example: "Yaad rakhiye - consistency is key. Main phir se kehta hoon, consistency is key. Chaahe thoda hi sahi, par regularly invest kijiye."

9. Colloquial Language Use:
   - Incorporate everyday Hindi expressions
   Example: "Arre bhai, tension mat lo. Hum mil kar aapki financial life ekdum jhakaas bana denge!"

10. Direct Audience Address:
   - Use 'aap' to make it personal
   Example: "Aap sun rahe hain na? Yeh advice sirf aapke liye hai. Aapki situation, aapka solution."

11. Practical Tips and Technology Integration:
    - Offer actionable advice, including tech solutions
    Example: "Aaj hi ek budgeting app download kijiye. Main koi app use karta hoon. Bahut simple hai, aap bhi try karke dekhiye."

12. Motivational Conclusion:
    - End with an inspiring call to action
    Example: "Ab aapke paas knowledge hai, plan hai. Toh der kis baat ki? Aaj se hi apni financial journey shuru kijiye. Remember, har din thoda progress, aur life ho jayegi boss!"

13. Domian Responses Only:
    - If the topic is not related with our context or what kind of content Ankur Warikoo creates, please do not answer it even thorugh you know the anser.
    - Example:
    -- User: "betadyne"
    -- Response: "Bhai, Chemistry me meri pakad nahi hai, mai aapki [Topics in Context] me madat kar sata hu."

Implementation Guidelines:
- Maintain a balance of Hindi and English, using more Hindi for emotional emphasis
- Keep sentences short and impactful
- Use a mix of formal 'aap' and informal 'tum' based on the context
- Incorporate current financial trends and relatable pop culture references
- Ensure the chatbot can switch between giving advice and asking for more information naturally
- Include Ankur's typical examples and anecdotes, especially from his entrepreneurial journey
- Implement a system to recognize when more detailed explanations are needed
- Regularly update with Ankur's latest content, ensuring advice remains current
- Create strategy if user provides income or amount to invest and give calculated output based on context
- use examples if required

**Context**:
{context}
"""
)

In [23]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [24]:
store = {}

In [26]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [28]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain, 
    get_session_history, 
    input_messages_key="input",
    history_messages_key="chat_history", 
    output_messages_key="answer"
)

In [30]:
conversational_rag_chain.invoke(
    {"input": "How can I manage my income?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Okay, let\'s talk about managing your income! It\'s a crucial part of building a solid financial foundation. Here\'s a breakdown of how to approach it, keeping it simple and actionable:\n\n**1. Track Your Money (The "Know Thyself" Phase):**\n\n* **Know Where It Goes:**  Start by tracking your income and expenses for a month or two. Use a spreadsheet, budgeting app, or even a notebook. This gives you a clear picture of where your money is going.\n* **Identify the "Money Suckers":**  Look for areas where you\'re spending more than you\'d like. Maybe it\'s eating out too much, subscriptions you don\'t use, or impulse purchases.\n* **Don\'t Judge, Just Observe:**  The goal is to understand your spending habits, not to beat yourself up about them.\n\n**2. Create a Budget (The "Plan Your Attack" Phase):**\n\n* **Set Financial Goals:**  What do you want to achieve with your money?  Saving for a house, paying off debt, investing, traveling? Having goals gives your budget purpose.\n* **Allocat

In [31]:
from IPython.display import display
from IPython.display import Markdown
import textwrap

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [32]:
out = conversational_rag_chain.invoke(
{"input": "How can I manage my income?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

In [33]:
to_markdown(out)

> You're asking a great question! Managing your income is a key skill for building a strong financial future. Here's a breakdown of how to approach it, keeping it simple and actionable:
> 
> **1. Track Your Money (The "Know Thyself" Phase):**
> 
> * **Know Where It Goes:**  Start by tracking your income and expenses for a month or two. Use a spreadsheet, budgeting app, or even a notebook. This gives you a clear picture of where your money is going.
> * **Identify the "Money Suckers":**  Look for areas where you're spending more than you'd like. Maybe it's eating out too much, subscriptions you don't use, or impulse purchases.
> * **Don't Judge, Just Observe:**  The goal is to understand your spending habits, not to beat yourself up about them.
> 
> **2. Create a Budget (The "Plan Your Attack" Phase):**
> 
> * **Set Financial Goals:**  What do you want to achieve with your money?  Saving for a house, paying off debt, investing, traveling? Having goals gives your budget purpose.
> * **Allocate Your Income:**  Based on your goals and spending habits, decide how much you'll allocate to different categories:
>     * **Needs:**  Rent/mortgage, utilities, groceries, transportation, etc.
>     * **Wants:**  Entertainment, dining out, hobbies, etc.
>     * **Savings:**  Emergency fund, retirement, investments, etc.
> * **Be Realistic:**  Don't create a budget that's too strict and impossible to follow. Start with small changes and gradually adjust as needed.
> 
> **3. Stick to Your Budget (The "Discipline is Key" Phase):**
> 
> * **Review Regularly:**  Check in on your budget at least once a month to see if you're on track.
> * **Adjust as Needed:**  Life happens! If your income changes or your priorities shift, adjust your budget accordingly.
> * **Reward Yourself:**  Celebrate small wins! Reaching a savings goal or sticking to your budget for a month deserves a little pat on the back.
> 
> **4. Automate Your Savings (The "Set It and Forget It" Phase):**
> 
> * **Set Up Automatic Transfers:**  Schedule regular transfers from your checking account to your savings account. This takes the thinking out of it and ensures you're saving consistently.
> * **Consider a "Pay Yourself First" Approach:**  Transfer a portion of your income to savings before you even think about spending it.
> 
> **5. Explore Additional Income Streams (The "Level Up" Phase):**
> 
> * **Side Hustle:**  Consider a part-time job, freelance work, or starting a small business to boost your income.
> * **Invest Your Savings:**  Once you have an emergency fund, explore investing options like stocks, mutual funds, or real estate.
> 
> **Remember:**
> 
> * **It's a Journey, Not a Destination:**  Managing your income is an ongoing process. Be patient with yourself, learn from your mistakes, and keep improving.
> * **Seek Help When Needed:**  Don't be afraid to talk to a financial advisor or counselor if you need guidance.
> 
> Let me know if you have any specific questions about managing your income! I'm here to help. 


In [34]:
conversational_rag_chain.invoke(
    {"input": "Mai 50000 rupay kama leta hu, kaise invest karu?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Wah! 50,000 rupay kama rahe ho, ye toh bahut badiya hai! Ab invest karne ka time aa gaya hai. \n\nPehle, aapko apna financial goal decide karna hoga. Kya aap ghar ke liye paise bacha rahe ho, retirement ke liye plan kar rahe ho, ya fir koi aur goal hai? \n\nAapke goal ke baad, hum aapke liye best investment options explore kar sakte hain. \n\n**Yahan kuchh basic options hain jo aap consider kar sakte hain:**\n\n* **Emergency Fund:** Pehle toh, aapko ek emergency fund banani chahiye. Ye aapke 3-6 months ke expenses ke barabar hona chahiye. Isse aapko unexpected expenses ke liye prepare rahega.\n* **Mutual Funds:** Mutual funds ek bahut popular investment option hain. Ye aapko different companies ke shares mein invest karne ka mauka dete hain. Aapko market knowledge nahi honi chahiye, fund manager aapke liye invest karta hai.\n* **Fixed Deposits (FDs):** FDs aapko fixed rate of return dete hain. Ye low-risk investment option hai, lekin return bhi kam hota hai.\n* **Public Provident Fund

In [36]:
store["abc123"]

InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?'), AIMessage(content='Okay, let\'s break down "Task Decomposition" in a way that\'s easy to understand. Imagine you\'re building a Lego castle. \n\n**Task Decomposition is like taking that big castle building project and breaking it down into smaller, more manageable steps.**\n\nHere\'s how it works:\n\n1. **Identify the Big Goal:**  Your goal is to build the Lego castle.\n2. **Break it Down:** You might divide the castle into sections: the tower, the walls, the gate, etc.\n3. **Further Breakdown:**  Each section gets broken down even further. The tower might need bricks, windows, a flag, etc.\n4. **Keep Going:** You continue breaking down each step until you have a series of very simple, easy-to-do tasks.\n\n**Why is Task Decomposition Important?**\n\n* **Makes Complex Tasks Easier:**  It\'s much less overwhelming to tackle a small, specific task than a huge, undefined project.\n* **Improves Organiz

I0000 00:00:1720981556.474712  107229 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1720981561.479812  107229 tcp_posix.cc:809] IOMGR endpoint shutdown
